In [1]:
import pandas as pd
from fredapi import Fred
import os
from openpyxl import load_workbook
from typing import List, Tuple, Union
from pathlib import Path
from dateutil.relativedelta import relativedelta

In [2]:
api="958ccd9c67808caf9f941367daf6e812"
fred = Fred(api_key=api)

In [3]:
monthly_tickers = [
    "SAHMREALTIME",
    "RECPROUSM156N",
    "PCEDGC96",
    'PCENDC96',
    "PCESC96",
    "RSAFS",
    "DSPIC96",
    "PSAVERT",
    "TOTALSA",
    "REVOLSL",
    "NONREVSL",
    "DGORDER",
    'ADXDNO',
    "INDPRO",
    "TCU",
    "HOUST",
    "PERMIT",
    "HSN1F",
    "EXHOSLUSM495S",
    "PAYEMS",
    "ADPMNUSNERSA",
    "UNRATE",
    "U6RATE",
    "CIVPART",
    "EMRATIO",
    "JTSJOR",
    "JTSHIR",
    "JTSTSR",
    "AWHAETP",
    "CPIAUCSL",
    "CPILFESL",
    "PPIFIS",
    "PCEPI",
    "PCEPILFE",
    "UMCSENT",
    "T5YIFR",
    'CES0500000003',
    "AHETPI",
    "CSUSHPINSA",
    "DTWEXBGS",
    "IQ",
    "IR",
    'BOPTEXP',
    'BOPTIMP',
    'BOPSTB'
]

quarterly_tickers = [
    "GDPC1",
    "NGDPSAXDCUSQ",
    "GDPNOW",
    "PCECC96",
    "PNFIC1",
    "PRFIC1",
    "GCE",
    "ECIWAG",
    'PRS85006092'
]

weekly_tickers = [
    "ICSA",
    "CCSA",
    "MORTGAGE30US"
]

daily_tickers = [
    "T10YIE",
    "DTWEXBGS",
    "FEDFUNDS",
    "DGS2",
    "DGS5",
    "DGS10",
    "DBAA"
]


In [4]:

# Fetch Data and Align Index
def Fetch(ticker_list, fred, freq):
    values_dfs = []
    dates_dfs = []

    # Set Frequency
    f = str(freq).strip().upper()
    if f in ("MONTHLY", "M"):
        period_code = "M"
    elif f in ("QUARTERLY", "Q"):
        period_code = "Q"
    elif f in ("WEEKLY", "W"):
        period_code = "W"
    elif f in ("ANNUAL", "A", "Y"):
        period_code = "A"
    elif f in ("DAILY", "D"):
        period_code = "D"
    else:
        raise ValueError(f"Unsupported frequency: {freq}")

    for ticker in ticker_list:
        try:
            s = fred.get_series(ticker)
        except Exception as e:
            print(f"Warning: couldn't fetch {ticker}: {e}")
            continue

        s.index = pd.to_datetime(s.index)
        df = s.reset_index()
        df.columns = ["release_date", ticker]

        if period_code == "D":
            df["Time"] = df["release_date"].dt.floor("D")
        else:
            df["Time"] = df["release_date"].dt.to_period(period_code).dt.to_timestamp()

        grouped = df.groupby("Time").agg({ticker: "last", "release_date": "last"})
        values_dfs.append(grouped[[ticker]])
        dates_dfs.append(grouped[["release_date"]].rename(columns={"release_date": ticker}))

    if not values_dfs:
        return pd.DataFrame(), pd.DataFrame()

    values = pd.concat(values_dfs, axis=1, sort=True)
    dates = pd.concat(dates_dfs, axis=1, sort=True)
    values.index.name = "Time"
    dates.index.name = "Time"

    return values, dates

monthly_values, monthly_dates = Fetch(monthly_tickers, fred, "M")
quarterly_values, quarterly_dates = Fetch(quarterly_tickers, fred, "Q")
weekly_values, weekly_dates = Fetch(weekly_tickers, fred, "W")
daily_values, daily_dates = Fetch(daily_tickers, fred, "D")

In [5]:
monthly_values

,SAHMREALTIME,RECPROUSM156N,PCEDGC96,PCENDC96,PCESC96,RSAFS,DSPIC96,PSAVERT,TOTALSA,REVOLSL,...,T5YIFR,CES0500000003,AHETPI,CSUSHPINSA,DTWEXBGS,IQ,IR,BOPTEXP,BOPTIMP,BOPSTB
Time,,,,,,,,,,,,,,,,,,,,,
1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-01,0.17,0.34,2113.9,3522.8,10863.2,723033.0,18036.2,5.0,16.042,1300321.06,...,2.27,36.31,31.26,331.671,119.8269,152.4,140.7,279650.0,338736.0,26631.0
2025-07-01,0.10,0.64,2152.2,3528.1,10888.2,727414.0,18077.3,4.8,16.896,1311491.90,...,2.32,36.43,31.34,331.127,122.1088,152.8,141.0,280464.0,358775.0,25565.0
2025-08-01,0.13,0.96,2171.4,3547.1,10910.9,732010.0,18097.2,4.6,16.802,1305533.90,...,2.35,36.53,31.46,NaN,120.6028,153.2,141.4,NaN,NaN,NaN


WindowsPath('C:/Users/hp/Desktop/Duke Research Opportunity/Prof. Aguilar Macro Dashboard/MacroDashBoard')

In [16]:
# to csv
# find project root relative to this script
# Save the data table to the folder "Raw Data"
base_dir = Path.cwd().parent.parent
print(base_dir)
raw_location = base_dir / "Raw Data"
raw_location.mkdir(parents=True, exist_ok=True)

monthly_values.to_csv(raw_location / "M_Values.csv")
monthly_dates.to_csv(raw_location / "M_Dates.csv")
quarterly_values.to_csv(raw_location / "Q_Values.csv")
quarterly_dates.to_csv(raw_location / "Q_Dates.csv")
weekly_values.to_csv(raw_location / "W_Values.csv")
weekly_dates.to_csv(raw_location / "W_Dates.csv")
daily_values.to_csv(raw_location / "D_Values.csv")
daily_dates.to_csv(raw_location / "D_Dates.csv")

C:\Users\hp\Desktop\Duke Research Opportunity\Prof. Aguilar Macro Dashboard\MacroDashBoard


In [36]:
# GetData from the csv files
def NormalizeFreq(code: str) -> str:
    """Return canonical single-letter code: 'A','Q','M','W','D' or raise."""
    fm = {
        ("MONTHLY", "M"): "M",
        ("QUARTERLY", "Q"): "Q",
        ("WEEKLY", "W"): "W",
        ("ANNUAL", "A", "Y"): "A",
        ("DAILY", "D"): "D",
    }
    code_up = code.upper()
    for keys, val in fm.items():
        if code_up in keys:
            return val
    raise ValueError(f"Unsupported frequency: {code}")

def GetData(ticker: str, freq: str, n_lags: int = 4):
    period_code=NormalizeFreq(freq)
    base = base_dir / "Raw Data"
    dates_path = os.path.join(base, f"{period_code}_Dates.csv")
    values_path = os.path.join(base, f"{period_code}_Values.csv")

    dates = pd.read_csv(dates_path,index_col=0, parse_dates=True)
    values = pd.read_csv(values_path,index_col=0, parse_dates=True)

    if ticker not in values.columns:
        raise KeyError(f"Ticker '{ticker}' not found in values file.")

    last_idx = values[ticker].last_valid_index()
    last_pos = values.index.get_loc(last_idx)
    latest_date = dates[ticker].iloc[last_pos]
    value_col = values[ticker]
    return latest_date, value_col

def GetLevel(ticker: str, freq: str, n_lags: int = 4):
    latest_date, value_col = GetData(ticker, freq, n_lags)
    last_idx = value_col.last_valid_index()
    last_pos = value_col.index.get_loc(last_idx)
    start_pos = max(0, last_pos - n_lags)
    latest_values = value_col.iloc[start_pos: last_pos + 1].tolist()
    return latest_date, latest_values

In [60]:
def GetDelta(ticker: str, freq: str, agg_freq: str, n_lags: int = 4, pct: bool = True) -> Tuple[str, pd.Series]:
    """
    Return (latest_date, delta_series) where delta_series is the same index as value_col
    containing differences (level diffs) or percent changes depending on `pct`.

    - `freq` is the native frequency of the data stored (e.g., "M" or "MONTHLY").
    - `agg_freq` is the aggregation period for the delta (e.g., "A" for year-over-year).
    - If value_col has a DatetimeIndex, the function prefers date-aware shifts (DateOffset/relativedelta).
    - If value_col has a plain integer index, the function uses integer shifts where we can derive them.
    - Raises on unsupported conversions (fractional shifts).
    """
    # integer shifts between canonical freq units when treating the series as regular rows
    # meaning: how many rows apart is one agg_freq period when rows are at 'freq' resolution?
    # e.g. if rows are monthly ('M') and agg_freq is 'A' -> shift = 12
    _INT_SHIFT_TABLE = {
        ("M", "A"): 12,
        ("M", "Q"): 3,
        ("Q", "A"): 4,
        ("Q", "M"): None,  # fractional: 1 quarter != integer months when you treat rows as quarters
        ("W", "M"): None,  # ambiguous unless you assume 4 or 4.345
        ("D", "M"): None,
        # same-to-same:
        ("M", "M"): 1,
        ("Q", "Q"): 1,
        ("A", "A"): 1,
        ("W", "W"): 1,
        ("D", "D"): 1,
    }

    # normalize freq inputs
    freq_code = NormalizeFreq(freq)
    agg_code = NormalizeFreq(agg_freq)

    # Get Data
    latest_date, value_col = GetData(ticker, freq_code, n_lags)

    # Normal MoM if the freq is M
    if agg_code == freq_code:
        shift_arg = 1
        if pct:
            result = value_col.pct_change(shift_arg,fill_method=None) * 100
        else:
            result = value_col.diff(shift_arg,fill_method=None)
        return latest_date, result

    # Default index is datetime index,so this is the default way of calculating delta
    idx = value_col.index
    if isinstance(idx, pd.DatetimeIndex) or pd.api.types.is_datetime64_any_dtype(idx):

        # choose the calendar offset and use .shift(freq=...) to move values for subtraction
        if freq_code == "M" and agg_code == "A":
            offset = pd.DateOffset(months=12)
        elif freq_code == "M" and agg_code == "Q":
            offset = pd.DateOffset(months=3)
        elif freq_code == "Q" and agg_code == "A":
            offset = pd.DateOffset(months=12)
        elif freq_code == "Q" and agg_code == "M":
            raise ValueError("Unsupported conversion: quarter-index to month-based aggregation when using DatetimeIndex.")
        elif freq_code == "W" and agg_code == "A":
            offset = pd.DateOffset(weeks=52)
        elif freq_code == "D" and agg_code == "A":
            offset = pd.DateOffset(years=1)
        else:
            raise ValueError(f"Unsupported date-based conversion: {freq_code} -> {agg_code}")

        # Move the historical values forward so that prev[t] == value_col[t - offset]
        prev = value_col.shift(freq=offset)
        prev = prev.reindex(value_col.index)

        if pct:
            result = (value_col - prev) / prev * 100
        else:
            result = value_col - prev

        return latest_date, result

    # If not datetime index: try integer shift from table _INT_SHIFT_TABLE
    key = (freq_code, agg_code)
    shift = _INT_SHIFT_TABLE.get(key, None)
    if shift is None:
        if freq_code == "Q" and agg_code == "A":
            shift = 4
        elif freq_code == "M" and agg_code == "A":
            shift = 12
        elif freq_code == "M" and agg_code == "Q":
            shift = 3
        else:
            raise ValueError(f"Unsupported conversion for integer-indexed series: {freq_code} -> {agg_code}")

    # safe integer shift
    shift = int(shift)
    if pct:
        result = value_col.pct_change(shift,fill_method=None) * 100
    else:
        result = value_col.diff(shift,fill_method=None)

    return latest_date, result


In [62]:
latest_date, result=GetDelta('PCEDGC96', 'M', 'Y', 4, pct= True)

In [63]:
result

Time
1919-01-01         NaN
1919-02-01         NaN
1919-03-01         NaN
1919-04-01         NaN
1919-05-01         NaN
                ...   
2025-06-01    3.495716
2025-07-01    3.925829
2025-08-01    5.203488
2025-09-01         NaN
2025-10-01         NaN
Name: PCEDGC96, Length: 1282, dtype: float64

In [53]:
monthly_values

,SAHMREALTIME,RECPROUSM156N,PCEDGC96,PCENDC96,PCESC96,RSAFS,DSPIC96,PSAVERT,TOTALSA,REVOLSL,...,T5YIFR,CES0500000003,AHETPI,CSUSHPINSA,DTWEXBGS,IQ,IR,BOPTEXP,BOPTIMP,BOPSTB
Time,,,,,,,,,,,,,,,,,,,,,
1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-01,0.17,0.34,2113.9,3522.8,10863.2,723033.0,18036.2,5.0,16.042,1300321.06,...,2.27,36.31,31.26,331.671,119.8269,152.4,140.7,279650.0,338736.0,26631.0
2025-07-01,0.10,0.64,2152.2,3528.1,10888.2,727414.0,18077.3,4.8,16.896,1311491.90,...,2.32,36.43,31.34,331.127,122.1088,152.8,141.0,280464.0,358775.0,25565.0
2025-08-01,0.13,0.96,2171.4,3547.1,10910.9,732010.0,18097.2,4.6,16.802,1305533.90,...,2.35,36.53,31.46,NaN,120.6028,153.2,141.4,NaN,NaN,NaN


In [54]:
monthly_values.shift(freq=pd.DateOffset(months=12))

,SAHMREALTIME,RECPROUSM156N,PCEDGC96,PCENDC96,PCESC96,RSAFS,DSPIC96,PSAVERT,TOTALSA,REVOLSL,...,T5YIFR,CES0500000003,AHETPI,CSUSHPINSA,DTWEXBGS,IQ,IR,BOPTEXP,BOPTIMP,BOPSTB
Time,,,,,,,,,,,,,,,,,,,,,
1920-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1920-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1920-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1920-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1920-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-06-01,0.17,0.34,2113.9,3522.8,10863.2,723033.0,18036.2,5.0,16.042,1300321.06,...,2.27,36.31,31.26,331.671,119.8269,152.4,140.7,279650.0,338736.0,26631.0
2026-07-01,0.10,0.64,2152.2,3528.1,10888.2,727414.0,18077.3,4.8,16.896,1311491.90,...,2.32,36.43,31.34,331.127,122.1088,152.8,141.0,280464.0,358775.0,25565.0
2026-08-01,0.13,0.96,2171.4,3547.1,10910.9,732010.0,18097.2,4.6,16.802,1305533.90,...,2.35,36.53,31.46,NaN,120.6028,153.2,141.4,NaN,NaN,NaN


In [26]:
# --- Constants ---
INPUT_XLSX = 'C:\\Users\\hp\\Desktop\\Duke Research Opportunity\\Prof. Aguilar Macro Dashboard\\MacroDashBoard\\MacroDashboard\\Economic Dashboard V1-Template.xlsx'
OUTPUT_XLSX = 'C:\\Users\\hp\\Desktop\\Duke Research Opportunity\\Prof. Aguilar Macro Dashboard\\MacroDashBoard\\MacroDashboard\\Economic Dashboard V1-Template-UPDATE.xlsx'

def _to_python_datetime(dt: Union[pd.Timestamp, str, None]):
    """
    Convert pandas.Timestamp -> python datetime for openpyxl.
    If dt is None or unparseable, return it unchanged.
    """
    if isinstance(dt, pd.Timestamp):
        return dt.to_pydatetime()
    return dt

# --- Load workbook & sheet ---
wb = load_workbook(INPUT_XLSX)
ws = wb.active

# --- Row bounds ---
start_row = 3
max_row = ws.max_row

def write_panel_for_row(
    row: int,
    ticker_col: str,
    freq_col: str,
    date_col: str,
    present_col: str,
    lag_cols: List[str],
    n_lags: int = 4,
) -> None:
    """
    Write one row for a panel (left or right).
    - ticker_col, freq_col, date_col, present_col: column letters (e.g. "B", "E", "C", "F")
    - lag_cols: list of lag column letters in order [Lag1_col, Lag2_col, ...]
    - n_lags: number of lags expected (defaults to 4)
    """
    # read ticker and freq values from the worksheet
    raw_ticker = ws[f"{ticker_col}{row}"].value
    raw_freq = ws[f"{freq_col}{row}"].value

    # If ticker cell is empty -> skip row
    if raw_ticker is None or str(raw_ticker).strip() == "":
        return

    # normalize strings safely
    ticker = str(raw_ticker).strip()
    freq = str(raw_freq).strip() if raw_freq is not None else "M"

    # get data using your GetData function (assumed defined/imported)
    try:
        latest_date, recent_values = GetData(ticker, freq, n_lags=n_lags)
    except Exception as exc:
        # write error to date cell and skip writing values for this row
        ws[f"{date_col}{row}"].value = f"ERR: {exc}"
        return

    # recent_values expected chronological oldest ... most recent
    values = list(recent_values)

    # Ensure fixed length = n_lags + 1 by padding at front (older side) with None
    expected_len = n_lags + 1
    if len(values) < expected_len:
        pad_len = expected_len - len(values)
        values = [None] * pad_len + values

    # Present is last element, lag1 is second-last, etc.
    present_value = values[-1]
    lag_values = []
    for i in range(1, len(lag_cols) + 1):
        # i=1 -> lag1 => values[-2], general: values[-1 - i]
        idx = -1 - i
        # safe access: if index out of range, return None
        try:
            lag_values.append(values[idx])
        except IndexError:
            lag_values.append(None)

    # write date (convert pandas Timestamp -> python datetime for openpyxl)
    ws[f"{date_col}{row}"].value = _to_python_datetime(latest_date)

    # write present and lag values
    ws[f"{present_col}{row}"].value = present_value
    for col_letter, lag_value in zip(lag_cols, lag_values):
        ws[f"{col_letter}{row}"].value = lag_value


# --- Panel definitions (columns) ---
# Left panel: B = ticker, E = freq, C = Latest Date, F = Present, G,H,I,J = Lag1..Lag4
left_panel = {
    "ticker_col": "B",
    "freq_col": "E",
    "date_col": "C",
    "present_col": "F",
    "lag_cols": ["G", "H", "I", "J"],
}

# Right panel: M = ticker, P = freq, N = Latest Date, Q = Present, R,S,T,U = Lag1..Lag4
right_panel = {
    "ticker_col": "M",
    "freq_col": "P",
    "date_col": "N",
    "present_col": "Q",
    "lag_cols": ["R", "S", "T", "U"],
}

# --- Iterate rows and write panels ---
for r in range(start_row, max_row + 1):
    write_panel_for_row(r, **left_panel)
    write_panel_for_row(r, **right_panel)

# --- Save updated workbook ---
wb.save(OUTPUT_XLSX)
print(f"Updated workbook saved to: {OUTPUT_XLSX}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hp\\Desktop\\Duke Research Opportunity\\Prof. Aguilar Macro Dashboard\\MacroDashBoard\\MacroDashboard\\Economic Dashboard V1-Template.xlsx'